In [16]:
import sys
import numpy as np
from gensim.models.wrappers.fasttext import FastText as ft
from scipy.spatial import distance
import pandas as pd
import MeCab


def word2id(bow, word_id):
    for w in bow:
        if w not in word_id:
            word_id[w] = len(word_id)
    return word_id

def compute_tf(sentences, word_id):
    tf = np.zeros([len(sentences), len(word_id)])

    for i in range(len(sentences)):
        for w in sentences[i]:
            tf[i][word_id[w]] += 1
    return tf

def compute_df(sentences, word_id):
    df = np.zeros(len(word_id))

    for i in range(len(sentences)):
        exist = {}
        for w in sentences[i]:
            if w not in exist:
                df[word_id[w]] += 1
                exist[w] = 1
            else:
                continue
    return df

def compute_idf(sentences, word_id):
    idf = np.zeros(len(word_id))
    df = compute_df(sentences, word_id)

    for i in range(len(df)):
        idf[i] = np.log(len(sentences)/df[i]) + 1
    return idf

def compute_tfidf(sentences):
    word_id = {}

    for sent in sentences:
        word_id = word2id(sent, word_id)

    tf = compute_tf(sentences, word_id)
    idf = compute_idf(sentences, word_id)

    tf_idf = np.zeros([len(sentences), len(word_id)])

    for i in range(len(sentences)):
        tf_idf[i] = tf[i] * idf

    return tf_idf

def compute_cosine(v1, v2):

    return 1 - distance.cosine(v1, v2)

def sent2vec(bow, model_w):

    vector = np.zeros(100)
    N = len(bow)

    for b in bow:
        try:
            vector += model_w[b]
        except:
            continue

    vector = vector / float(N)

    return vector

def compute_word2vec(sentences):

    model = ft.load_fasttext_format("../Downloads/model")
    
    vector = np.zeros([len(sentences), 100])

    for i in range(len(sentences)):
        vector[i] = sent2vec(sentences[i], model_w)

    return vector

def lexrank(sentences, N, threshold, vectorizer):

    CosineMatrix = np.zeros([N, N])
    degree = np.zeros(N)
    L = np.zeros(N)

    if vectorizer == "tf-idf":
        vector = compute_tfidf(sentences)
    elif vectorizer == "word2vec":
        vector = compute_word2vec(sentences)

    # Computing Adjacency Matrix                                                                                                                                         
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = compute_cosine(vector[i], vector[j])
            if CosineMatrix[i,j] > threshold:
                CosineMatrix[i,j] = 1
                degree[i] += 1
            else:
                CosineMatrix[i,j] = 0

    # Computing LexRank Score                                                                                                                                            
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = CosineMatrix[i,j] / degree[i]

    L = PowerMethod(CosineMatrix, N, err_tol=10e-6)

    return L

def PowerMethod(CosineMatrix, N, err_tol):

    p_old = np.array([1.0/N]*N)
    err = 1

    while err > err_tol:
        err = 1
        p = np.dot(CosineMatrix.T, p_old)
        err = np.linalg.norm(p - p_old)
        p_old = p

    return p

def extractKeyword(text):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text).next
    keywords = []
    while node:
        if node.feature.split(",")[0] == "名詞":
            keywords.append(node.surface)
        node = node.next
    return keywords

In [5]:
import a

In [7]:
a??

In [9]:
b = Hoge()
b.hoge()

[ 0.  0.]


In [ ]:
class Lexrank():
    def __init__(self, sentences):
    
        self.sent = sentences
    
    def __tf_idf_(self):
        self.tmp = {}
        
        for i in self.sent:
            tmp = self.__word2id_()
        
    def compute_tfidf(sentences):
    word_id = {}

    for sent in sentences:
        word_id = word2id(sent, word_id)

    tf = compute_tf(sentences, word_id)
    idf = compute_idf(sentences, word_id)

    tf_idf = np.zeros([len(sentences), len(word_id)])

    for i in range(len(sentences)):
        tf_idf[i] = tf[i] * idf

    return tf_idf
    
    def fit(self,threshold,N=0,vectorizer='tf-idf'):
        if N == 0:
            self.N = len(self.sent)
        
        self.cos_matrix = np.zeros((self.N,self.N))
        self.__degree_ = np.zeros(self.N)
        self.score = np.zeros(self.N)
        
        if vectorizer == 'tf-idf':
            self.vector = self.__tf_idf_()
        else:
            sys.exit('Vectorizer Error Occured!')
        
        
            
            
    CosineMatrix = np.zeros([N, N])
    degree = np.zeros(N)
    L = np.zeros(N)

    if vectorizer == "tf-idf":
        vector = compute_tfidf(sentences)
    elif vectorizer == "word2vec":
        vector = compute_word2vec(sentences)

    # Computing Adjacency Matrix                                                                                                                                         
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = compute_cosine(vector[i], vector[j])
            if CosineMatrix[i,j] > threshold:
                CosineMatrix[i,j] = 1
                degree[i] += 1
            else:
                CosineMatrix[i,j] = 0

    # Computing LexRank Score                                                                                                                                            
    for i in range(N):
        for j in range(N):
            CosineMatrix[i,j] = CosineMatrix[i,j] / degree[i]

    L = PowerMethod(CosineMatrix, N, err_tol=10e-6)

    return L

In [19]:
kuchikomi = pd.read_csv('./test.csv',encoding="SHIFT-JIS",header=None)

In [20]:
tmp = kuchikomi[0].tolist()

In [102]:
stop = ['']
sentences = [extractKeyword(tmp[x],stop) for x in range(len(tmp))]
hoge = lexrank(sentences,21,0.10,'tf-idf')
for k in range(5):
    print(tmp[np.argsort(hoge)[::-1][k]], np.sort(hoge)[::-1][k])

インターンはたくさん参加しよう。 0.0833533165365
ケース面接の本は、何冊か読めばインターンまではとりあえず行けると思います。 0.0714011814499
インターン参加者の中には、なぜ自分が通ったのか疑問を感じる人も多く、もしかしたら面接官によってばらつきがあるのかもしれない。 0.0595734377148
インターン参加者の質は非常に高く、他の外資金融、外資コンサルのインターン参加者ばかりで、他のインターンを通じて知り合いだったものも多いようだ。 0.0595511320714
インターンの選考で重視されているのは、頭の良さと人柄だと思います。 0.0595125947791


In [132]:
stop = ['インターン', 'ビーシージー']
sentences = [extractKeyword(tmp[x],stop) for x in range(len(tmp))]
n = 300

k = np.random.rand(n)
score = np.zeros(n)
for i in range(n):
    hoge = lexrank(sentences,21,k[i],'tf-idf')
    score[i] = (np.sort(hoge)[::-1][0] - np.sort(hoge)[::-1][3]) / (np.sort(hoge)[::-1][0] - np.sort(hoge)[::-1][2])
hoge = lexrank(sentences,21,k[score.argmax()],'tf-idf')
for j in range(4):
    print(tmp[np.argsort(hoge)[::-1][j]], np.sort(hoge)[::-1][j])

ケース問題の対策にあたっては、「東大生の書いたフェルミ推定ノート」などケーススタディの参考書を一読することを勧めます。 0.047619047619
あまりにもガツガツ行き過ぎて周りから非難されるような人は、たとえ優秀であったとしてもBCGは厳しいかもしれない。 0.047619047619
１次面接でも２次面接でもケース問題をやり、いかにそこで論理的思考力があるかをアピールできるかにかかっています。 0.047619047619
3日間という短い期間であるものの、参加している学生の頭脳レベルがとても高く、学生界で最高峰の議論、グループワークを経験できるという意味で非常に刺激になると思います。 0.047619047619


In [142]:
hoge = lexrank(sentences,21,0.01,'tf-idf')
for j in range(4):
    print(tmp[np.argsort(hoge)[::-1][j]], np.sort(hoge)[::-1][j])

インターン参加者の中には、なぜ自分が通ったのか疑問を感じる人も多く、もしかしたら面接官によってばらつきがあるのかもしれない。 0.0756966416305
インターンに行く前はそういう風には考えられないとは思うが、インターンに受かったならば、BCGが出版している本をせめて一冊は読んでから、インターンに臨んでほしい。 0.0677295576006
インターンで得た仲間とは今でも選考情報について交換することがあり、将来的にも大きな財産になると思います。 0.0677288496378
3日間という短い期間であるものの、参加している学生の頭脳レベルがとても高く、学生界で最高峰の議論、グループワークを経験できるという意味で非常に刺激になると思います。 0.0637437746891


In [145]:
hoge = lexrank(sentences,21,0.02,'tf-idf')
for j in range(4):
    print(tmp[np.argsort(hoge)[::-1][j]], np.sort(hoge)[::-1][j])

インターン参加者の中には、なぜ自分が通ったのか疑問を感じる人も多く、もしかしたら面接官によってばらつきがあるのかもしれない。 0.0871781833651
インターンの選考で重視されているのは、頭の良さと人柄だと思います。 0.0769235922643
ケース面接の本は、何冊か読めばインターンまではとりあえず行けると思います。 0.0666675850699
インターンはたくさん参加しよう。 0.0615401832788


In [130]:
for j in range(4):
    print(tmp[np.argsort(hoge)[::-1][j]], np.sort(hoge)[::-1][j])

インターン参加者の中には、なぜ自分が通ったのか疑問を感じる人も多く、もしかしたら面接官によってばらつきがあるのかもしれない。 0.108529217486
BCGの選考だからと身構えることなく、能力については基礎的な論理的思考力と少しの慣れがあれば十分だと思います。 0.0697645565719
１次面接でも２次面接でもケース問題をやり、いかにそこで論理的思考力があるかをアピールできるかにかかっています。 0.0697585130913
エントリーシートに書いた内容などは面接では一切触れられないので、志望動機や論理的思考以外の自己PRはあまり見られていないかもしれません。 0.0620086611189


In [94]:
def extractKeyword(text,stopword=['']):
    tagger = MeCab.Tagger()
    tagger.parse('')
    node = tagger.parseToNode(text).next
    keywords = []
    hinshi = ['名詞','形容詞','動詞']
    hinshi2 = ['接尾','非自立']
    while node:
        tmp = node.feature.split(',')
        if tmp[0] in hinshi and tmp[1] not in hinshi2 and tmp[-1] not in stopword:
            if tmp[6] == '*' and node.surface not in keywords:
                keywords.append(node.surface)
                #print(keywords[-1], node.feature)
            elif tmp[6] not in keywords:
                keywords.append(tmp[6])
                #print(keywords[-1], node.feature)
            else:
                pass
        node = node.next
    return keywords

In [83]:
tmp[0]

'「頭のよさ」とは具体的には、いわゆる論理的思考力があることと、クリエイティビティのような頭の柔らかさがあることが求められていると思います。'

In [84]:
x = extractKeyword(tmp[0])

頭 名詞,一般,*,*,*,*,頭,アタマ,アタマ
よい 形容詞,自立,*,*,形容詞・アウオ段,ガル接続,よい,ヨ,ヨ
具体的 名詞,固有名詞,一般,*,*,*,具体的,グタイテキ,グタイテキ
論理的思考 名詞,固有名詞,一般,*,*,*,論理的思考,ロンリテキシコウ,ロンリテキシコー
ある 動詞,自立,*,*,五段・ラ行,基本形,ある,アル,アル
クリエイティビティ 名詞,一般,*,*,*,*,*
柔らかい 形容詞,自立,*,*,形容詞・アウオ段,ガル接続,柔らかい,ヤワラカ,ヤワラカ
求める 動詞,自立,*,*,一段,未然形,求める,モトメ,モトメ
思う 動詞,自立,*,*,五段・ワ行促音便,連用形,思う,オモイ,オモイ


In [81]:
import MeCab

tagger = MeCab.Tagger('-Ochasen')
tagger.parse('')
node = tagger.parseToNode(tmp[0]).next
key = []
while node:
    print(node.surface,node.feature)
    key.append(node.surface)
    node = node.next
print(key)

「 記号,括弧開,*,*,*,*,「,「,「
頭 名詞,一般,*,*,*,*,頭,アタマ,アタマ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
よ 形容詞,自立,*,*,形容詞・アウオ段,ガル接続,よい,ヨ,ヨ
さ 名詞,接尾,特殊,*,*,*,さ,サ,サ
」 記号,括弧閉,*,*,*,*,」,」,」
と 助詞,格助詞,引用,*,*,*,と,ト,ト
は 助詞,係助詞,*,*,*,*,は,ハ,ワ
具体的 名詞,固有名詞,一般,*,*,*,具体的,グタイテキ,グタイテキ
に 助詞,格助詞,一般,*,*,*,に,ニ,ニ
は 助詞,係助詞,*,*,*,*,は,ハ,ワ
、 記号,読点,*,*,*,*,、,、,、
いわゆる 連体詞,*,*,*,*,*,いわゆる,イワユル,イワユル
論理的思考 名詞,固有名詞,一般,*,*,*,論理的思考,ロンリテキシコウ,ロンリテキシコー
力 名詞,接尾,一般,*,*,*,力,リョク,リョク
が 助詞,格助詞,一般,*,*,*,が,ガ,ガ
ある 動詞,自立,*,*,五段・ラ行,基本形,ある,アル,アル
こと 名詞,非自立,一般,*,*,*,こと,コト,コト
と 助詞,並立助詞,*,*,*,*,と,ト,ト
、 記号,読点,*,*,*,*,、,、,、
クリエイティビティ 名詞,一般,*,*,*,*,*
の 助詞,連体化,*,*,*,*,の,ノ,ノ
よう 名詞,非自立,助動詞語幹,*,*,*,よう,ヨウ,ヨー
な 助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
頭 名詞,一般,*,*,*,*,頭,アタマ,アタマ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
柔らか 形容詞,自立,*,*,形容詞・アウオ段,ガル接続,柔らかい,ヤワラカ,ヤワラカ
さ 名詞,接尾,特殊,*,*,*,さ,サ,サ
が 助詞,格助詞,一般,*,*,*,が,ガ,ガ
ある 動詞,自立,*,*,五段・ラ行,基本形,ある,アル,アル
こと 名詞,非自立,一般,*,*,*,こと,コト,コト
が 助詞,格助詞,一般,*,*,*,が,ガ,ガ
求め 動詞,自立,*,*,一段,未然形,求める,モトメ,モトメ
られ 動詞,接尾,*,*,一段,連用形,られる,ラレ,ラレ
て 助詞,接続助詞,*,*,*,*,て,テ,テ
いる 動詞,非自立,*,*,一段,基本形,いる,イル,イル
と 助詞,格助詞,